In [2]:
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt

import copy # to copy lists for python2.7

try:
    from .individual_single_mode import Individual
except Exception: #ImportError
    from individual_single_mode import Individual

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
############################### EVALFLAG.PY ##########################################
## A method to evaluate the quality of flagging.

## Inputs : data, flags
## Output : some metric of quality.

## Test : Run tfcrop/rflag with diff params, saving flagversions
##           Extract the 2D flags (perhaps just from flagversions)

VIS_DIR = '/Users/bjuncklaus/Dropbox/Evolutionary/Data/datasets/'
VIS_FILENAME = 'FewScans_G55_Spw2_Scan50.ms'
# VIS_FILENAME = 'FewScans_G55_Spw6_Scan6_HanningSmooth.ms'
# VIS_FILENAME = 'Four_ants_3C286.ms_Spw9_Scan30.ms'
# VIS_FILENAME = 'G29_Spw0_Scan145.ms'
# VIS_FILENAME = 'G29_Spw0_Scan38.ms'
# VIS_FILENAME = 'G29_Spw7_Scan83.ms'
# TODO - put the % Flagged on the best fit
# VIS_FILENAME = 'FewScans_G55_Spw6_Scan52_HanningSmooth.ms'
# VIS = '/home/vega2/bmartins/datasets/FewScans_G55_Spw2_Scan4.ms'
# VIS = '/Users/bjuncklaus/Dropbox/Evolutionary/Data/datasets/FewScans_G55_Spw6_Scan4_HanningSmooth.ms'
VIS = VIS_DIR + VIS_FILENAME

In [1]:
def runtest(cmdlist=[]):
    ## G55 dataset
    ## scans 4,6 are 3C286
    ## scans 50,52 are G55 SNR
    ## spws 6 and 11 have good representative RFI.


    # vis = '/home/vega2/bmartins/datasets/FewScans_G55_Spw7_Scan4.ms'

    # vis = '../Data/Four_ants_3C286.ms'

    spw = '0'
    scan = '4'
    vname = 'v1'

    ## Run the flagging
    flagdata(vis=VIS, mode='unflag', flagbackup=False)

    if (not cmdlist):
    #     tfcrop_params = "spw='" + spw + "' scan='" + scan + "'" + cmdlist[0]
    #     extend_params = "spw='" + spw + "' scan='" + scan + "'" + cmdlist[1]
    #     cmdlist = [tfcrop_params, extend_params]
    # else:
        # cmdlist = ["spw='"+spw+"' scan='"+scan+"' mode='tfcrop' maxnpieces=4 freqcutoff=3.0 usewindowstats='sum' " ,
        #              "spw='"+spw+"' scan='"+scan+"' mode='extend' growaround=True growtime=60.0" ]
        cmdlist = ["' mode='tfcrop'"]  # default value

    print()
    print("CMDLIST:", cmdlist)

    flagdata(vis=VIS, mode='list', inpfile=cmdlist, flagbackup=False)
    # flagmanager(vis=vis, mode='save', versionname=vname)

    ## Read the flags
    # flagmanager(vis=vis, mode='restore', versionname=vname)
    dat = getvals(col='DATA', vis=VIS)
    flag = getvals(col='FLAG', vis=VIS)

    # plotit(dat, flag)

    flag_percentage = np.sum(flag) / (1.0 * np.prod(flag.shape)) * 100.0
    print('% Flagged : ', flag_percentage)
    print('VIS : ', VIS_FILENAME)

    score = calcquality(dat, flag)
    if (math.isnan(score)):
        return float("inf"), flag_percentage

    return score, flag_percentage